In [2]:
import pandas as pd

In [3]:
SE=pd.read_csv("Sortedevents.csv")
SB=pd.read_csv("Sortedbattery.csv")


In [4]:
SE.head()

,VID,TYPE,VAL,TS_parsed
0,026af092-a01e-4b0a-af38-f459725dabc6,IGNITION,UNKNOWN,2023-01-01 16:57:56.699000+00:00
1,026af092-a01e-4b0a-af38-f459725dabc6,IGNITION,UNKNOWN,2023-01-01 17:49:22.838000+00:00
2,026af092-a01e-4b0a-af38-f459725dabc6,IGNITION,UNKNOWN,2023-01-03 14:04:28.196000+00:00
3,026af092-a01e-4b0a-af38-f459725dabc6,IGNITION,UNKNOWN,2023-01-06 03:15:29.994000+00:00
4,026af092-a01e-4b0a-af38-f459725dabc6,IGNITION,UNKNOWN,2023-01-06 07:45:11.710000+00:00


In [5]:
SB.head()

,VID,BATTERY_LEVEL,TS_parsed
0,026af092-a01e-4b0a-af38-f459725dabc6,98.0,2023-01-01 02:36:17.442000+00:00
1,026af092-a01e-4b0a-af38-f459725dabc6,98.0,2023-01-01 02:36:47.885000+00:00
2,026af092-a01e-4b0a-af38-f459725dabc6,NaN,2023-01-01 02:36:47.886000+00:00
3,026af092-a01e-4b0a-af38-f459725dabc6,98.0,2023-01-01 02:37:18.561000+00:00
4,026af092-a01e-4b0a-af38-f459725dabc6,97.0,2023-01-01 02:37:48.866000+00:00


In [10]:
import pandas as pd
import numpy as np
from datetime import timedelta

def find_closest_battery_level(event_row, battery_df, time_window=300):
    """
    Find the closest battery level for an event row based on VID and TS_parsed within time_window seconds.
    """
    vid = event_row['VID']
    event_ts = event_row['TS_parsed']
    
    # Filter battery_df for the same VID
    battery_subset = battery_df[battery_df['VID'] == vid]
    
    if battery_subset.empty:
        return np.nan, np.nan  # No matching VID found
    
    # Calculate absolute time difference
    time_diffs = abs(battery_subset['TS_parsed'] - event_ts).dt.total_seconds()
    
    # Find the index of the minimum time difference within the time window
    valid_diffs = time_diffs[time_diffs <= time_window]
    
    if valid_diffs.empty:
        return np.nan, np.nan  # No timestamp within 300 seconds
    
    min_diff_idx = valid_diffs.idxmin()
    closest_battery_level = battery_subset.loc[min_diff_idx, 'BATTERY_LEVEL']
    closest_ts = battery_subset.loc[min_diff_idx, 'TS_parsed']
    
    return closest_battery_level, closest_ts

def create_events_final(battery_df, events_df, time_window=300):
    """
    Create EVENTS_Final DataFrame by matching battery levels from battery_df to events_df.
    """
    # Ensure TS_parsed is in datetime format with explicit format for ISO 8601
    battery_df['TS_parsed'] = pd.to_datetime(battery_df['TS_parsed'], format='ISO8601', utc=True)
    events_df['TS_parsed'] = pd.to_datetime(events_df['TS_parsed'], format='ISO8601', utc=True)
    
    # Initialize EVENTS_Final DataFrame
    events_final = events_df[['VID', 'TS_parsed', 'TYPE', 'VAL']].copy()
    events_final['Battery_Level'] = np.nan
    events_final['Matched_TS'] = np.nan  # For verification
    
    # Apply the function to find closest battery level
    for idx, row in events_final.iterrows():
        battery_level, matched_ts = find_closest_battery_level(row, battery_df, time_window)
        events_final.at[idx, 'BATTERY_LEVEL'] = battery_level
        events_final.at[idx, 'Matched_TS'] = matched_ts
    
    # Rename TS_parsed to TS
    events_final.rename(columns={'TS_parsed': 'TS'}, inplace=True)
    
    return events_final

def verify_events_final(events_final, battery_df, time_window=300):
    """
    Verify the correctness of EVENTS_Final DataFrame.
    """
    print("Verification Report:")
    print("-------------------")
    
    # Check for NaN battery levels
    nan_count = events_final['BATTERY_LEVEL'].isna().sum()
    total_rows = len(events_final)
    print(f"Rows with missing Battery_Level: {nan_count} ({nan_count/total_rows*100:.2f}%)")
    
    # Check if VIDs match
    invalid_vids = 0
    for idx, row in events_final.iterrows():
        if pd.notna(row['BATTERY_LEVEL']):
            matched_ts = row['Matched_TS']
            vid = row['VID']
            battery_subset = battery_df[(battery_df['TS_parsed'] == matched_ts) & (battery_df['VID'] == vid)]
            if battery_subset.empty:
                invalid_vids += 1
    
    print(f"Rows with mismatched VIDs: {invalid_vids}")
    
    # Check if time differences are within 300 seconds
    time_violations = 0
    for idx, row in events_final.iterrows():
        if pd.notna(row['BATTERY_LEVEL']) and pd.notna(row['Matched_TS']):
            time_diff = abs(row['TS'] - row['Matched_TS']).total_seconds()
            if time_diff > time_window:
                time_violations += 1
    
    print(f"Rows with time difference > {time_window} seconds: {time_violations}")
    
    # Summary
    if nan_count == 0 and invalid_vids == 0 and time_violations == 0:
        print("All checks passed! The EVENTS_Final DataFrame is correctly generated.")
    else:
        print("Issues found. Please review the data.")

In [11]:
events_final = create_events_final(SB, SE)
verify_events_final(events_final, SB)

C:\Users\Abhiram Sharma\AppData\Local\Temp\ipykernel_20416\897959107.py:50: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2023-01-01 16:57:56.699000+00:00' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  events_final.at[idx, 'Matched_TS'] = matched_ts


KeyboardInterrupt: 

In [14]:
VID="026af092-a01e-4b0a-af38-f459725dabc6"

In [ ]:
from datetime import datetime, timedelta
import pytz

# Original timestamp string
timestamp_str = '2023-01-02 21:08:03+00:00'

# Parse it as a timezone-aware datetime object
timestamp = datetime.fromisoformat(timestamp_str)

# Subtract 300 seconds
new_timestamp = timestamp - timedelta(seconds=300)

print(new_timestamp.isoformat())


In [15]:
filtered_df = SE[SE["VID"] == VID]
filtereddict_df = SB[SB["VID"] == VID]
filtered_df['Battery'] = np.nan
filtereddict_df = filtereddict_df.drop('VID', axis=1)
# Iterate through the filtered rows
for index, row in filtered_df.iterrows():
    target1=row['TS_parsed'] - timedelta(seconds=300)
    target2=row['TS_parsed'] + timedelta(seconds=300)
    for index2,row2 in filtereddict_df.iterrows():
        if (row2['TS_parsed'] < target2) & (row2['TS_parsed'] > target1):
            row['Battery']=row2["BATTERY_LEVEL"]
    

C:\Users\Abhiram Sharma\AppData\Local\Temp\ipykernel_20416\3180938703.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Battery'] = np.nan

KeyboardInterrupt



In [19]:
def vid_check(VID):
    from datetime import timedelta
    import numpy as np
    filtered_df = SE[SE["VID"] == VID]
    filtereddict_df = SB[SB["VID"] == VID]
    # Assuming filtered_df and filtereddict_df are already created as in your code
    filtered_df['Battery'] = np.nan
    filtereddict_df = filtereddict_df.drop('VID', axis=1)
    
    # Iterate and update
    for index, row in filtered_df.iterrows():
        target1 = row['TS_parsed'] - timedelta(seconds=300)
        target2 = row['TS_parsed'] + timedelta(seconds=300)
        
        # Find matching rows in the second DataFrame
        matching_rows = filtereddict_df[
            (filtereddict_df['TS_parsed'] > target1) & 
            (filtereddict_df['TS_parsed'] < target2)
        ]
        
        if not matching_rows.empty:
            # For example, assign the first matched battery level
            filtered_df.loc[index, 'Battery'] = matching_rows.iloc[0]['BATTERY_LEVEL']
    return filtered_df


In [18]:
filtered_df.head()

,VID,TYPE,VAL,TS_parsed,Battery
0,026af092-a01e-4b0a-af38-f459725dabc6,IGNITION,UNKNOWN,2023-01-01 16:57:56.699000+00:00,NaN
1,026af092-a01e-4b0a-af38-f459725dabc6,IGNITION,UNKNOWN,2023-01-01 17:49:22.838000+00:00,NaN
2,026af092-a01e-4b0a-af38-f459725dabc6,IGNITION,UNKNOWN,2023-01-03 14:04:28.196000+00:00,62.0
3,026af092-a01e-4b0a-af38-f459725dabc6,IGNITION,UNKNOWN,2023-01-06 03:15:29.994000+00:00,NaN
4,026af092-a01e-4b0a-af38-f459725dabc6,IGNITION,UNKNOWN,2023-01-06 07:45:11.710000+00:00,59.0


In [22]:
unique_vids = SE['VID'].unique().tolist()

In [25]:
alldf = []
for each in unique_vids:
    newdf=vid_check(each)
    alldf.append(newdf)
final_df = pd.concat(alldf, ignore_index= True)

C:\Users\Abhiram Sharma\AppData\Local\Temp\ipykernel_20416\2887326148.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Battery'] = np.nan
C:\Users\Abhiram Sharma\AppData\Local\Temp\ipykernel_20416\2887326148.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Battery'] = np.nan
C:\Users\Abhiram Sharma\AppData\Local\Temp\ipykernel_20416\2887326148.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_index

In [26]:
final_df.describe()

,Battery
count,3159.000000
mean,62.804495
std,21.939589
min,1.900000
25%,47.800000
50%,67.000000
75%,78.400000
max,110.000000


In [27]:
final_df.head()

,VID,TYPE,VAL,TS_parsed,Battery
0,026af092-a01e-4b0a-af38-f459725dabc6,IGNITION,UNKNOWN,2023-01-01 16:57:56.699000+00:00,NaN
1,026af092-a01e-4b0a-af38-f459725dabc6,IGNITION,UNKNOWN,2023-01-01 17:49:22.838000+00:00,NaN
2,026af092-a01e-4b0a-af38-f459725dabc6,IGNITION,UNKNOWN,2023-01-03 14:04:28.196000+00:00,62.0
3,026af092-a01e-4b0a-af38-f459725dabc6,IGNITION,UNKNOWN,2023-01-06 03:15:29.994000+00:00,NaN
4,026af092-a01e-4b0a-af38-f459725dabc6,IGNITION,UNKNOWN,2023-01-06 07:45:11.710000+00:00,59.0
